In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('May_17_Agreement')
page.get_parse()

en.wikipedia.org (parse) May_17_Agreement
May 17 Agreement (en) data
{
  iwlinks: <list(1)> https://fr.wikipedia.org/wiki/Fattal
  pageid: 3671727
  parsetree: <str(9794)> <root><template><title>pp</title><part><n...
  requests: <list(1)> parse
  title: May 17 Agreement
  wikibase: Q321851
  wikidata_url: https://www.wikidata.org/wiki/Q321851
  wikitext: <str(7829)> {{pp|small=yes}}{{short description|1983 p...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "May_17_Agreement",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('May 17 Agreement')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1948 Arab-Israeli War
1949 Armistice Agreements
1982 Lebanon War
2000 Camp David Summit
2002 Arab League summit
2010–2011 Israeli–Palestinian peace talks
2013–2014 Israeli–Palestinian peace talks
Abraham Accords
Agreement on Disengagement between Israel and Syria
Amal Movement
American trusteeship proposal for Palestine
Amine Gemayel
Anglo-French Declaration
Annapolis Conference
Arab League
Arab world
Arab–Israeli normalization
Bachir Gemayel
Bahrain–Israel normalization agreement
Balfour Declaration
Beilin–Abu Mazen agreement
Bevin Plan
Blockade of the Gaza Strip
Camp David Accords
Camp David Accords (1978)
Cease-fire
Cedar Revolution
Churchill White Paper
Confessionalism (politics)
Damascus
Damascus Protocol
Declaration to the Seven
Druze
Egypt
Egypt–Israel peace treaty
Ehud Barak
Faisal–Weizmann Agreement
Gaza–Jericho Agreement
Geneva Initiative (2003)
Golan Heights
Guerrilla warfare
Hezbollah
Hussein el-Husseini
ISBN (identifier)
Israel
Israeli Army
Israeli disengagement fro

In [4]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod
        
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Arab-Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 2000 Camp David Summit
en.wikipedia.org (parse) 2002 Arab League summit
en.wikipedia.org (parse) 2010–2011 Israeli–Palestinian peace talks
en.wikipedia.org (parse) 2013–2014 Israeli–Palestinian peace talks
en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Amal Movement


[[Nabih Berri]]
[[Musa al-Sadr]]<br /> [[Hussein el-Husseini]]
-----------------
name:  ['Amal Movement']
founders:  Nabih Berri,Musa al-Sadr,Hussein el-Husseini,
ideology:  ['Populism', 'Conservatism']
position:  ['Centre-right']
START
name:  Amal Movement
name ar:  حركة أمل 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D8%A3%D9%85%D9%84
END
START
ideology:  Populism
ideology ar:  شعبوية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B4%D8%B9%D8%A8%D9%88%D9%8A%D8%A9
ideology:  Conservatism
ideology ar:  سياسة محافظة 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9_%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9
END
START
position:  Centre-right
position ar:  وسط اليمين 
position ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%B3%D8%B7_%D8%A7%D9%84%D9%8A%D9%85%D9%8A%D9%86
END
START
founder:  Nabih Berri
founder ar:  نبيه بري 
founder ar link:  https://ar.wikipedia.org/wiki/%D9%86%D8%A8%D9%8A%D9%87_%D8%A8%D8%B1%D9%8A
Cannot translate phras

en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Anglo-French Declaration
en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab world
en.wikipedia.org (imageinfo) File:Arab world location map.svg
en.wikipedia.org (parse) Arab–Israeli normalization
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...
en.wikipedia.org (parse) Bahrain–Israel normalization agreement
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) Balfour Declaration
en.wikipedia.org (imageinfo) File:Balfour declaration unmarked.jpg
en.wikipedia.org (parse) Beilin–Abu Mazen agreement
en.wikipedia.org (parse) Bevin Plan
en.wikipedia.org (parse) Blockade of the Gaza Strip
en.wikipedia.org (parse) Camp David Accords
en.wikipedia.

Cannot translate phrase `Hussein el-Husseini` to arabic
END
[[Hassan Nasrallah|Sayyed Hassan Nasrallah]]
-----------------
name:  ['Hezbollah']
founders:  Sayyed Hassan Nasrallah,
ideology:  ['Antisemitism', 'Anti-imperialism', 'Pan-Islamism', 'Anti-West', 'Islamic nationalism', 'Unbulleted list', 'Anti-Zionism', 'Shia Jihad', 'Khomeinism']
position:  -
START
name:  Hezbollah
name ar:  حزب الله 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%84%D9%87
END
START
ideology:  Antisemitism
ideology ar:  معاداة السامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B3%D8%A7%D9%85%D9%8A%D8%A9
ideology:  Anti-imperialism
ideology ar:  معاداة الاستعمارية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%A7%D8%B3%D8%AA%D8%B9%D9%85%D8%A7%D8%B1%D9%8A%D8%A9
ideology:  Pan-Islamism
ideology ar:  وحدة إسلامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%AD%D8

en.wikipedia.org (parse) Hussein el-Husseini
en.wikipedia.org (imageinfo) File:Husseini1.jpg
en.wikipedia.org (parse) ISBN (identifier)
en.wikipedia.org (imageinfo) File:EAN-13-ISBN-13.svg
en.wikipedia.org (parse) Israel
en.wikipedia.org (parse) Israeli Army
en.wikipedia.org (parse) Israeli disengagement from Gaza
en.wikipedia.org (parse) Israel–Jordan peace treaty
en.wikipedia.org (imageinfo) File:Flickr - Government Press Offic...
en.wikipedia.org (parse) Israel–Morocco normalization agreement
en.wikipedia.org (imageinfo) File:American-Israeli delegation vis...
en.wikipedia.org (parse) Israel–Sudan normalization agreement
en.wikipedia.org (parse) Israel–United Arab Emirates normalizatio...
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) John Kerry Parameters
en.wikipedia.org (parse) Kamel Asaad
en.wikipedia.org (imageinfo) File:Kamel Asaad 2.jpg
en.wikipedia.org (parse) Khartoum Resolution
en.wikipedia.org (parse) Lausanne Conference of 1

[[Samir Geagea]]
[[Bachir Gemayel]]
-----------------
name:  ['Lebanese Forces']
founders:  Samir Geagea,Bachir Gemayel,
ideology:  ['Liberal conservatism', 'Christian nationalism', 'National conservatism', 'Lebanese nationalism']
position:  ['Right-wing']
START
name:  Lebanese Forces
name ar:  القوات اللبنانية 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D9%88%D8%A7%D8%AA_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
ideology:  Liberal conservatism
ideology ar:  محافظة ليبرالية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9_%D9%84%D9%8A%D8%A8%D8%B1%D8%A7%D9%84%D9%8A%D8%A9
ideology:  Christian nationalism
ideology ar:  قومية مسيحية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%B3%D9%8A%D8%AD%D9%8A%D8%A9
ideology:  National conservatism
ideology ar:  محافظة وطنية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9_%D9%88%D8%B7%D9%86%D9%8A%D8%

en.wikipedia.org (parse) Lebanon
en.wikipedia.org (parse) List of Middle East peace proposals
en.wikipedia.org (parse) London Conference of 1939
en.wikipedia.org (parse) London Conference of 1946–1947
en.wikipedia.org (parse) Madrid Conference of 1991
en.wikipedia.org (parse) McMahon–Hussein Correspondence
en.wikipedia.org (parse) Military occupation
en.wikipedia.org (parse) Morrison–Grady Plan
en.wikipedia.org (parse) Oslo Accords
en.wikipedia.org (parse) Palestine Liberation Organization


[[Mahmoud Abbas]]
-----------------
name:  ['Palestine Liberation Organization']
founders:  Mahmoud Abbas,
ideology:  ['Palestinian nationalism']
position:  -
START
name:  Palestine Liberation Organization
name ar:  منظمة التحرير الفلسطينية 
name ar link:  https://ar.wikipedia.org/wiki/%D9%85%D9%86%D8%B8%D9%85%D8%A9_%D8%A7%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D8%A7%D9%84%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
Cannot translate phrase `Mahmoud Abbas` to arabic
END


en.wikipedia.org (parse) Palestinian autonomy talks
en.wikipedia.org (parse) Peel Commission
en.wikipedia.org (imageinfo) File:PeelMap.png
en.wikipedia.org (parse) President of Lebanon
en.wikipedia.org (imageinfo) File:Michel Aoun, February 2020 (cro...
en.wikipedia.org (parse) Prime Minister of Israel
en.wikipedia.org (imageinfo) File:Naftali Bennett official portrait.jpg
en.wikipedia.org (parse) Road map for peace
en.wikipedia.org (parse) San Remo conference
en.wikipedia.org (imageinfo) File:San Remo Conference 1920.JPG
en.wikipedia.org (parse) Sharm El Sheikh Memorandum
en.wikipedia.org (parse) Sharm El Sheikh Summit of 2005
en.wikipedia.org (parse) Shebaa Farms
en.wikipedia.org (parse) Siege of Beirut
en.wikipedia.org (parse) Sovereignty
en.wikipedia.org (parse) Sykes–Picot Agreement
en.wikipedia.org (imageinfo) File:MPK1-426 Sykes Picot Agreement ...
en.wikipedia.org (parse) Syria
en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Cedar Revolut

skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli diplomacy
skipping Template talk:Arab–Israeli diplomacy


In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Amal Movement,"Populism,Conservatism",Centre-right,"Nabih Berri,Musa al-Sadr,Hussein el-Husseini,"
1,Hezbollah,"Antisemitism,Anti-imperialism,Pan-Islamism,Ant...",-,"Sayyed Hassan Nasrallah,"
2,Lebanese Forces,"Liberal conservatism,Christian nationalism,Nat...",Right-wing,"Samir Geagea,Bachir Gemayel,"
3,Palestine Liberation Organization,Palestinian nationalism,-,"Mahmoud Abbas,"


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,حركة أمل,"شعبوية ,سياسة محافظة",وسط اليمين,نبيه بري
1,حزب الله,"معاداة السامية ,معاداة الاستعمارية ,وحدة إسلام...",-,حسن نصر الله
2,القوات اللبنانية,"محافظة ليبرالية ,قومية مسيحية ,محافظة وطنية ,ق...",يمينية,"سمير جعجع ,بشير الجميل"
3,منظمة التحرير الفلسطينية,قومية فلسطينية,-,


In [10]:
datasets_dir = 'datasets_updated/1982-1984/May_17_Agreement/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Arab-Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 2000 Camp David Summit
en.wikipedia.org (parse) 2002 Arab League summit
en.wikipedia.org (parse) 2010–2011 Israeli–Palestinian peace talks
en.wikipedia.org (parse) 2013–2014 Israeli–Palestinian peace talks
en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg


-----------------
name:  ['Amine Gemayel']
native_name:  ['أمين الجميل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  ['Lebanese']
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Anglo-French Declaration
en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab world
en.wikipedia.org (imageinfo) File:Arab world location map.svg
en.wikipedia.org (parse) Arab–Israeli normalization
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...


-----------------
name:  ['ar', 'بشير الجميّل', 'Bachir Gemayel', 'lang']
native_name:  -
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  -
name ar:  بشير الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
name ar:  بشير الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  -


en.wikipedia.org (parse) Bahrain–Israel normalization agreement
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) Balfour Declaration
en.wikipedia.org (imageinfo) File:Balfour declaration unmarked.jpg
en.wikipedia.org (parse) Beilin–Abu Mazen agreement
en.wikipedia.org (parse) Bevin Plan
en.wikipedia.org (parse) Blockade of the Gaza Strip
en.wikipedia.org (parse) Camp David Accords
en.wikipedia.org (imageinfo) File:Close up of Menahem Begin, Jimm...
en.wikipedia.org (parse) Camp David Accords (1978)
en.wikipedia.org (imageinfo) File:Close up of Menahem Begin, Jimm...
en.wikipedia.org (parse) Cease-fire
en.wikipedia.org (parse) Cedar Revolution
en.wikipedia.org (parse) Churchill White Paper
en.wikipedia.org (parse) Confessionalism (politics)
en.wikipedia.org (parse) Damascus
en.wikipedia.org (parse) Damascus Protocol
en.wikipedia.org (parse) Declaration to the Seven
en.wikipedia.org (parse) Druze
en.wikipedia.org (parse) Egypt
en.wikipedia.org

-----------------
name:  ['كامل بك الأسعد', 'Kamel Bek El Assaad']
native_name:  -
nickname:  -
religion:  -
party:  ['Lebanese Social Democratic Party']
nationality:  ['Lebanese']
Cannot translate phrase `Kamel Asaad` to arabic
name ar:  -1
START
political party:  Lebanese Social Democratic Party
political party ar:  الحزب الاشتراكي الديمقراطي اللبناني (مترجمه)
political party ar link:  -1
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  -1
religion ar:  ['الحزب الاشتراكي الديمقراطي اللبناني (مترجمه)']
party ar:  ['Lebanese Social Democratic Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Khartoum Resolution
en.wikipedia.org (parse) Lausanne Conference of 1949
en.wikipedia.org (parse) Lebanese Army
en.wikipedia.org (parse) Lebanese Civil War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
en.wikipedia.org (parse) Lebanese Forces
en.wikipedia.org (parse) Lebanon
en.wikipedia.org (parse) List of Middle East peace proposals
en.wikipedia.org (parse) London Conference of 1939
en.wikipedia.org (parse) London Conference of 1946–1947
en.wikipedia.org (parse) Madrid Conference of 1991
en.wikipedia.org (parse) McMahon–Hussein Correspondence
en.wikipedia.org (parse) Military occupation
en.wikipedia.org (parse) Morrison–Grady Plan
en.wikipedia.org (parse) Oslo Accords
en.wikipedia.org (parse) Palestine Liberation Organization
en.wikipedia.org (parse) Palestinian autonomy talks
en.wikipedia.org (parse) Peel Commission
en.wikipedia.org (imageinfo) File:PeelMap.png
en.wikipedia.org (parse) President of Lebanon
en.wikipedia.org (imageinfo) File:Michel

-----------------
name:  ['Walid Jumblatt']
native_name:  ['ar', 'وليد جنبلاط', 'lang']
nickname:  -
religion:  -
party:  ['Progressive Socialist Party']
nationality:  -
name ar:  وليد جنبلاط 
START
political party:  Progressive Socialist Party
political party ar:  الحزب التقدمي الاشتراكي 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%AA%D9%82%D8%AF%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A
END
name ar:  وليد جنبلاط 
religion ar:  ['الحزب التقدمي الاشتراكي ']
party ar:  ['Progressive Socialist Party']
nationality ar:  -


en.wikipedia.org (parse) White Paper of 1939
en.wikipedia.org (imageinfo) File:1939 White Paper cmd 6019.djvu
en.wikipedia.org (parse) Wye River Memorandum


skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli diplomacy
skipping Template talk:Arab–Israeli diplomacy


In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,"(Amine Gemayel,)",Kataeb Party,Lebanese,-
1,"(ar,بشير الجميّل,Bachir Gemayel,lang,)",Kataeb Party,-,-
2,"(كامل بك الأسعد,Kamel Bek El Assaad,)",Lebanese Social Democratic Party,Lebanese,-
3,"(Walid Jumblatt,)",Progressive Socialist Party,-,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,لبناني (مترجمه),حزب الكتائب اللبنانية,أمين الجميل
1,-,-,حزب الكتائب اللبنانية,بشير الجميل
2,-,لبناني (مترجمه),الحزب الاشتراكي الديمقراطي اللبناني (مترجمه),-1
3,-,-,الحزب التقدمي الاشتراكي,وليد جنبلاط


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [17]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
    
                    df_locations_en = df_locations_en.append({
                        'Name': ','.join(name) if isinstance(name, list) and len(name) > 1 else name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Arab-Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 2000 Camp David Summit
en.wikipedia.org (parse) 2002 Arab League summit
en.wikipedia.org (parse) 2010–2011 Israeli–Palestinian peace talks
en.wikipedia.org (parse) 2013–2014 Israeli–Palestinian peace talks
en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Anglo-French Declaration
en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab world
en.wikipedia.org (imageinfo) File:Arab world location map.svg
en.wikipedia.o

-----------------
name:  ['Damascus']
subdivision_type:  ['Country']
subdivision_name:  ['Syria']
name ar:  دمشق 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Syria
subdivision_name ar:  سوريا 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B3%D9%88%D8%B1%D9%8A%D8%A7
END


en.wikipedia.org (parse) Damascus Protocol
en.wikipedia.org (parse) Declaration to the Seven
en.wikipedia.org (parse) Druze
en.wikipedia.org (parse) Egypt


-----------------
name:  ['Egypt']
subdivision_type:  -
subdivision_name:  -
name ar:  مصر 


en.wikipedia.org (parse) Egypt–Israel peace treaty
en.wikipedia.org (imageinfo) File:Sadat Carter Begin handshake (c...
en.wikipedia.org (parse) Ehud Barak
en.wikipedia.org (imageinfo) File:Ehud Barak 2016 - Herzliya Conf...
en.wikipedia.org (parse) Faisal–Weizmann Agreement
en.wikipedia.org (parse) Gaza–Jericho Agreement
en.wikipedia.org (parse) Geneva Initiative (2003)
en.wikipedia.org (parse) Golan Heights


...

-----------------
name:  ['Syria']
subdivision_type:  -
subdivision_name:  -
name ar:  سوريا 


en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstrators.jpg
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Taboo
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) Tripartite Declaration of 1950
en.wikipedia.org (parse) Trump peace plan
en.wikipedia.org (parse) United Nations
en.wikipedia.org (parse) United Nations General Assembly Resolution 194
en.wikipedia.org (parse) United Nations Partition Plan for Palestine
en.wikipedia.org (imageinfo) File:UN Palestine Partition Versions...
en.wikipedia.org (parse) United Nations Security Council Resoluti...
en.wikipedia.org (imageinfo) File:LocationLebanon.svg
en.wikipedia.org (parse) United Nations Security Council Resoluti...
en.wikipedia.org (imageinfo) File:Aus Emb bomb2.png
en.wikipedia.org (parse) United Nations Security Council Resoluti...
en.wikipedia.org (imageinfo) File:Lebanon history UNIFIL.jpg
en.wikipedia.org (parse) United Nations

-----------------
name:  ['the United States']
subdivision_type:  -
subdivision_name:  -
name ar:  الولايات المتحدة 


en.wikipedia.org (parse) Walid Jumblatt
en.wikipedia.org (imageinfo) File:Walid Jumblatt 6C2.jpg
en.wikipedia.org (parse) White Paper of 1939
en.wikipedia.org (imageinfo) File:1939 White Paper cmd 6019.djvu
en.wikipedia.org (parse) Wye River Memorandum


skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli diplomacy
skipping Template talk:Arab–Israeli diplomacy


In [19]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,[Damascus],Country,Syria
1,[Egypt],-,-
2,Golan Heights,Status,"group,Internationally recognized as Syrian ter..."
3,[Israel],-,-
4,[Lebanon],-,-
5,[Syria],-,-
6,[the United States],-,-


In [20]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,سوريا,بلد,دمشق
1,-,-,مصر
2,"مجموعة (توضيح) ,معترف بها دوليًا باسم تيري الس...",حالة (مترجمه),هضبة الجولان
3,-,-,إسرائيل
4,-,-,لبنان
5,-,-,سوريا
6,-,-,الولايات المتحدة


In [21]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')